In [ ]:
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
df.head()

* сколько и какие вакцины используются в мире
* сколько всего вакцинированных в каждой из стран
* какой процент от общего населения вакцинирован в каждой стране (информацию по численности населения в странах необходимо найти дополнительно).  
  
При ответе на 2 и 3 вопросы сделайте график/диаграмму с помощью библиотеки matplotlib

### 1. сколько и какие вакцины используются в мире

In [ ]:
unique_vacc = set()
for joined_names in df['vaccines'].unique():
    vacc_names = joined_names.split(',')
    for vacc_name in vacc_names:
        vacc_name = vacc_name.strip()
        unique_vacc.add(vacc_name)

        
pprint(len(unique_vacc))
pprint(unique_vacc)

### 2. сколько всего вакцинированных в каждой из стран

In [ ]:
gb_country = df.groupby('country')[['total_vaccinations']].max()

fig = plt.figure(figsize=(15, 5))
ax = fig.add_axes([0,0,1,1])

ax.bar(gb_country.index, gb_country['total_vaccinations'])
fig.autofmt_xdate(rotation=90) 
plt.show()

### 3. какой процент от общего населения вакцинирован в каждой стране (информацию по численности населения в странах необходимо найти дополнительно).  

In [ ]:
country_data = pd.read_csv(
    '/kaggle/input/undata-country-profiles/country_profile_variables.csv',
    index_col='country', usecols=['country','Population in thousands (2017)']
).rename(columns={'Population in thousands (2017)':'population'})
country_data = country_data * 1000

country_rename_mapping = {
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Faroe Islands':'Faeroe Islands',
    'Russian Federation':'Russia',
    'United States of America': 'United States'
}
country_data = country_data.rename(index=country_rename_mapping)

country_data.head()

In [ ]:
missing_countries = {
    'England': 55.62 * (10 ** 6),
    'Guernsey': 67_052,
    'Jersey': 105_600,
    'Northern Cyprus': 326_000,
    'Northern Ireland': 1.885 * (10 ** 6),
    'Scotland': 5.46 * (10 ** 6),
    'Wales': 3.15 * (10 ** 6),
}
missing_countries_df = pd.DataFrame.from_dict(missing_countries, orient='index', columns=['population']).astype(int)
full_country_data = pd.concat([missing_countries_df, country_data], verify_integrity=True)

In [ ]:
gb_country['population'] = full_country_data.loc[list(gb_country.index), :]
gb_country['% vaccinated'] = (gb_country['total_vaccinations'] / gb_country['population']) * 100
gb_country.head()

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_axes([0,0,1,1])

ax.bar(gb_country.index, gb_country['% vaccinated'])
fig.autofmt_xdate(rotation=90) 
plt.show()